In [6]:
import numpy as np
from PIL import Image
from bitarray import bitarray
import math

In [7]:
def file_To_Bits(path):
    with open(path,'r+b') as file:
        file_bytes=file.read()
    
    bits=bitarray()
    bits.frombytes(file_bytes)
    return bits.tolist()

def bits_To_File(path,output_bits):
    outputFile_bits=bitarray(output_bits)

    with open(path,'wb') as file:
        outputFile_bits.tofile(file)
    

In [8]:
def embedding_Audio(audio_path, carrier_image_path, output_image):
    audio_bits=file_To_Bits(audio_path)

    image=Image.open(carrier_image_path).convert("RGB")
    decoded_image=np.array(image,dtype=np.uint8)

    # By reshaping the 3D array into a 2D array with .reshape(-1, 3), we are effectively:
    #     Flattening the pixel structure while maintaining the RGB grouping.
    #     Instead of explicitly storing RGB values in a nested array for each pixel, we make each row of the 2D array represent a single pixels RGB values.
    # 3D Array (Original Shape):
    # Structure: (rows, cols, 3)

    # For an image with 2 rows and 2 columns (4 pixels total), it looks like:
    # [
    #     [ [R, G, B], [R, G, B] ],  # First row
    #     [ [R, G, B], [R, G, B] ]   # Second row
    # ]

    # When reshaped, it becomes:
    # [
    #     [R, G, B],  # Pixel 1
    #     [R, G, B],  # Pixel 2
    #     [R, G, B],  # Pixel 3
    #     [R, G, B]   # Pixel 4
    # ]
    # The grid structure is removed, and all pixels are stored in a single, continuous array.
    # Each row represents one pixel, and the columns are its R, G, and B values.
    rows, col, channels=decoded_image.shape
    decoded_image=decoded_image.reshape(-1,3)

    image_bit_capacity=decoded_image.shape[0]*3 # rows * col * 3, The number of LSB bits available, not the entire image bits
    print(f"Carrier image dimensions: {rows}x{col}")
    print(f"Image bit capacity: {image_bit_capacity} bits")

    audio_bits_capacity= 32 +len(audio_bits)

    if audio_bits_capacity>image_bit_capacity:
        difference=math.ceil(math.sqrt(audio_bits_capacity/image_bit_capacity))
        image_resize=Image.open(carrier_image_path)
        image_resize=image_resize.resize((image.width*difference,image.height*difference))
        image_resize.save(carrier_image_path)

        raise ValueError(f"Image resized to {image_resize.width}x{image_resize.height}")

    audio_length_bits = bitarray()
    audio_length_bits.frombytes(len(audio_bits).to_bytes(4, byteorder='big'))


    audio_payload_bits = audio_length_bits.tolist() + audio_bits 
    print(f"Audio payload capacity: {audio_payload_bits} bits")

    # Embeding Audio bits into the LSBs of the carrier image RGB pixel values
    bit_index=0
    for pixel in decoded_image:
        for channel in range(3):
            if bit_index < len(audio_payload_bits):

                pixel[channel]= (pixel[channel] & 0xFE) | audio_payload_bits[bit_index]

                # The above line sets the LSB of the carrier image with the audio
                # 1. It first clears the clears the LSB of the pixel's channel value in the image using the & with 0xFE
                    # 0xFE in binary is 11111110, so performing a bitwise AND with it ensures that the LSB of the channel value becomes 0, while leaving all other bits unchanged.
                    # It clears the least significant bit of the current channel.

                # 2. | payload_bits[bit_index]
                    # Then we set the LSB to the value of the current bit from payload_bits (0 or 1).
                    # The bitwise OR (|) operation ensures that the cleared LSB is replaced by the current bit.
                # This effectively embeds one bit of the payload_bits into the LSB of the current channel.
                bit_index+=1
            else:
                break
        if bit_index>len(audio_payload_bits):
            break

        
    encoded_image = decoded_image.reshape(rows, col, 3)
    embedded_Image = Image.fromarray(encoded_image, "RGB")
    embedded_Image.save(output_image)
    print(f"Embedded image saved as {output_image}")





In [9]:
def audio_extractor(embedded_image_path, output_audio_path):
    image=Image.open(embedded_image_path).convert("RGB")
    decoded_image=np.array(image,dtype=np.uint8)

    decoded_image=decoded_image.reshape(-1,3)
    extracted_bits=[]
    # We extract all the LSBs of the image
    for pixel in decoded_image:
        for channel in range(3):
            extracted_bits.append(pixel[channel] & 1)

    # audio_length=extracted_bits[:32]   
    # audio_length = int(bitarray(audio_length, endian='big').tobytes().hex(), 16)
    length_bits = bitarray(extracted_bits[:32], endian='big')
    audio_length = int.from_bytes(length_bits.tobytes(), byteorder='big')

    print(f"Audio length: {audio_length} bits")
    

    bits_To_File(output_audio_path,extracted_bits[32:(audio_length+32)])
    print(f"Recovered Audio file saved as {output_audio_path}")

In [ ]:
audio_path="./Files/Audio/Dreams pt II.mp3"
carrier_image_path="./Files/Images/Dreams pt II.png"
output_image_path="./Files/Embedded/Dreams pt II.png"

embedding_Audio(audio_path,carrier_image_path,output_image_path)



In [ ]:
extracted_audio="./Files/Embedded/Dreams pt II.mp3"

audio_extractor(output_image_path,extracted_audio)